In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leaf-dataset-vimal/dataset/valid/Tomato___Late_blight/Tomato___Late_blight_0008.JPG
/kaggle/input/leaf-dataset-vimal/dataset/valid/Tomato___Late_blight/Tomato___Late_blight_0005.JPG
/kaggle/input/leaf-dataset-vimal/dataset/valid/Tomato___Late_blight/Tomato___Late_blight_0036.JPG
/kaggle/input/leaf-dataset-vimal/dataset/valid/Tomato___Late_blight/Tomato___Late_blight_0006.JPG
/kaggle/input/leaf-dataset-vimal/dataset/valid/Tomato___Late_blight/Tomato___Late_blight_0030.JPG
/kaggle/input/leaf-dataset-vimal/dataset/valid/Tomato___Late_blight/Tomato___Late_blight_0048.JPG
/kaggle/input/leaf-dataset-vimal/dataset/valid/Tomato___Late_blight/Tomato___Late_blight_0001.JPG
/kaggle/input/leaf-dataset-vimal/dataset/valid/Tomato___Late_blight/Tomato___Late_blight_0002.JPG
/kaggle/input/leaf-dataset-vimal/dataset/valid/Tomato___Late_blight/Tomato___Late_blight_0034.JPG
/kaggle/input/leaf-dataset-vimal/dataset/valid/Tomato___Late_blight/Tomato___Late_blight_0018.JPG
/kaggle/input/leaf-d

In [6]:
# ================================================================
# LGNM - MAX SPEED VERSION (Early Stop at 96% Accuracy)
# ================================================================

import os, cv2, time, random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from torchvision import transforms, datasets
from torchvision.models import mobilenet_v3_large

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report


# =========================
# CONFIG
# =========================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

TRAIN_DIR = "/kaggle/input/leaf-dataset-vimal/dataset/train"
VALID_DIR = "/kaggle/input/leaf-dataset-vimal/dataset/valid"
TEST_DIR  = "/kaggle/input/leaf-dataset-vimal/dataset/test"

OUT_DIR = "/kaggle/working/fast_visuals"
MODEL_PATH = "/kaggle/working/LGNM_fast_model.pth"
os.makedirs(OUT_DIR, exist_ok=True)

CFG = {
    "image_size": 160,
    "batch_size": 32,
    "epochs": 50,          # but early stop will cut it short
    "lr": 2e-4,
    "weight_decay": 1e-5,
    "num_graph_nodes": 32,
    "num_workers": 2,
    "target_acc": 0.96     # STOP here
}


# ============================================================
# FAST GRAPH EXTRACTOR
# ============================================================
class FastGraphExtractor:
    def __init__(self, num_nodes=32):
        self.num_nodes = num_nodes

    def extract(self, tensor_img):
        img = (tensor_img.permute(1,2,0).numpy()*255).astype(np.uint8)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        edges = cv2.Canny(gray, 40, 110)

        pts = np.column_stack(np.where(edges > 0))
        h, w = edges.shape

        if len(pts) < 1:
            pts = np.random.randint(0, min(h,w), (self.num_nodes,2))
        if len(pts) < self.num_nodes:
            pad = np.random.randint(0, min(h,w), (self.num_nodes-len(pts),2))
            pts = np.vstack([pts, pad])

        idx = np.linspace(0, len(pts)-1, self.num_nodes).astype(int)
        pts = pts[idx]

        feat = img[pts[:,0], pts[:,1]] / 255.0  # 3 features

        feat = torch.tensor(feat, dtype=torch.float32)
        edges_list = []
        for i in range(self.num_nodes):
            d = np.linalg.norm(pts - pts[i], axis=1)
            nn_i = np.argsort(d)[1:4]
            for j in nn_i:
                edges_list.append([i,j])

        if len(edges_list)==0:
            edge_index = torch.zeros((2,0), dtype=torch.long)
        else:
            edge_index = torch.tensor(edges_list).t().contiguous()

        return {"x":feat, "edge_index":edge_index}


# ============================================================
# DATASET
# ============================================================
class LGNMDataset(Dataset):
    def __init__(self, root, transform, g_ex):
        self.tform = transform
        self.raw = transforms.ToTensor()
        self.folder = datasets.ImageFolder(root, transform=self.tform)
        self.samples = self.folder.samples
        self.g = g_ex

    def __len__(self):
        return len(self.folder)

    def __getitem__(self, idx):
        img_t, label = self.folder[idx]
        path,_ = self.samples[idx]

        raw = Image.open(path).convert("RGB")
        raw = self.raw(raw.resize((CFG["image_size"],CFG["image_size"])))

        graph = self.g.extract(raw)

        return img_t, graph, label


def collate_fn(batch):
    imgs, graphs, labels = zip(*batch)
    imgs = torch.stack(imgs)
    labels = torch.tensor(labels)

    xs=[]; es=[]; b=[]; offset=0
    for i,g in enumerate(graphs):
        x = g["x"]; e = g["edge_index"]
        xs.append(x)
        if e.numel()>0:
            es.append(e + offset)
        b.append(torch.full((x.size(0),), i))
        offset += x.size(0)

    X = torch.cat(xs,0)
    B = torch.cat(b,0)
    if len(es)>0: E = torch.cat(es,1)
    else: E = torch.zeros((2,0),dtype=torch.long)

    return imgs, {"x":X, "edge_index":E, "batch":B}, labels


# ============================================================
# SUPER FAST GCN
# ============================================================
class FastGCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(3, 32)
        self.l2 = nn.Linear(32, 64)
        self.relu = nn.ReLU()

    def forward(self, g):
        x = g["x"]
        ei = g["edge_index"]
        b = g["batch"]

        h = self.relu(self.l1(x))

        N = h.size(0)
        if ei.numel()==0:
            return torch.zeros((b.max()+1,64),device=x.device)

        src,dst = ei
        agg = torch.zeros_like(h)
        agg.index_add_(0, dst, h[src])

        deg = torch.zeros((N,),device=x.device)
        deg.index_add_(0, dst, torch.ones_like(dst,dtype=torch.float))
        deg = deg.clamp_min(1).unsqueeze(1)

        h2 = self.relu(self.l2(agg/deg))

        num_g = int(b.max())+1
        pooled = torch.zeros((num_g,64), device=x.device)
        pooled.index_add_(0, b, h2)

        c = torch.zeros((num_g,),device=x.device)
        c.index_add_(0,b,torch.ones_like(b,dtype=torch.float))
        pooled /= c.unsqueeze(1)

        return pooled


# ============================================================
# MODEL: MobileNetV3 + FastGCN
# ============================================================
class LGNM_FAST(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        base = mobilenet_v3_large(weights="DEFAULT")
        self.cnn = base.features
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.proj = nn.Linear(960, 256)

        self.gnn = FastGCN()

        self.fuse = nn.Linear(256+64, 128)
        self.cls = nn.Linear(128, num_classes)

    def forward(self, imgs, g):
        with autocast():
            f = self.cnn(imgs)
            f = self.pool(f).view(f.size(0),-1)
            f = self.proj(f)

        with torch.cuda.amp.autocast(enabled=False):
            gfeat = self.gnn({
                "x":g["x"].float().to(f.device),
                "edge_index":g["edge_index"].to(f.device),
                "batch":g["batch"].to(f.device)
            })

        if gfeat.size(0)!=f.size(0):
            gfeat = gfeat.mean(0,keepdim=True).expand(f.size(0),-1)

        z = torch.cat([f,gfeat],1)
        z = F.relu(self.fuse(z))
        return self.cls(z)


# ============================================================
# TRANSFORMS
# ============================================================
train_tf = transforms.Compose([
    transforms.Resize((CFG["image_size"],CFG["image_size"])),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test_tf = transforms.Compose([
    transforms.Resize((CFG["image_size"],CFG["image_size"])),
    transforms.ToTensor(),
])


# ============================================================
# LOAD DATA
# ============================================================
g_ex = FastGraphExtractor(CFG["num_graph_nodes"])

train_ds = LGNMDataset(TRAIN_DIR, train_tf, g_ex)
valid_ds = LGNMDataset(VALID_DIR, test_tf, g_ex)
test_ds  = LGNMDataset(TEST_DIR,  test_tf, g_ex)

train_loader = DataLoader(train_ds, CFG["batch_size"], True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_ds, CFG["batch_size"], False, collate_fn=collate_fn)
test_loader  = DataLoader(test_ds,  CFG["batch_size"], False, collate_fn=collate_fn)

classes = train_ds.folder.classes
print("Classes:", classes)


# ============================================================
# TRAINING WITH 96% EARLY STOP
# ============================================================
model = LGNM_FAST(len(classes)).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])
criterion = nn.CrossEntropyLoss()
scaler = GradScaler()

best_acc = 0

for ep in range(CFG["epochs"]):

    # ---------------- TRAIN ----------------
    model.train()
    preds=[]; labs=[]; loss_sum=0

    for imgs,g,y in train_loader:
        imgs = imgs.to(DEVICE)
        y = y.to(DEVICE)

        optimizer.zero_grad()
        with autocast():
            out = model(imgs,g)
            loss = criterion(out,y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        loss_sum += loss.item()*imgs.size(0)
        preds.extend(out.argmax(1).cpu().numpy())
        labs.extend(y.cpu().numpy())

    tr_acc = accuracy_score(labs,preds)

    # ---------------- VALID ----------------
    model.eval()
    v_preds=[]; v_labs=[]

    with torch.no_grad():
        for imgs,g,y in valid_loader:
            imgs=imgs.to(DEVICE)
            out=model(imgs,g)
            v_preds.extend(out.argmax(1).cpu().numpy())
            v_labs.extend(y.numpy())

    val_acc = accuracy_score(v_labs,v_preds)
    print(f"Epoch {ep+1}: Train Acc={tr_acc:.4f}, Val Acc={val_acc:.4f}")

    # SAVE BEST
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({"model":model.state_dict(),"classes":classes}, MODEL_PATH)
        print("  → Best model updated.")

    # ================================
    # EARLY STOP ON ACC ≥ 96%
    # ================================
    if tr_acc >= CFG["target_acc"] and val_acc >= CFG["target_acc"]:
        print("EARLY STOP TRIGGERED — 96% REACHED!")
        break
    if val_acc >= CFG["target_acc"]:
        print("EARLY STOP — VALIDATION ACC ≥ 96%")
        break


# ============================================================
# FINAL EVALUATION
# ============================================================
ckpt = torch.load(MODEL_PATH, map_location=DEVICE)
model.load_state_dict(ckpt["model"])

def eval_loader(loader):
    p=[]; l=[]
    with torch.no_grad():
        for imgs,g,y in loader:
            imgs=imgs.to(DEVICE)
            out=model(imgs,g)
            p.extend(out.argmax(1).cpu().numpy())
            l.extend(y.numpy())
    return accuracy_score(l,p), f1_score(l,p,average="weighted")

train_acc,_ = eval_loader(train_loader)
val_acc,_ = eval_loader(valid_loader)
test_acc,test_f1 = eval_loader(test_loader)

print("\n===== FINAL SCORES =====")
print("Train Accuracy:", train_acc)
print("Valid Accuracy:", val_acc)
print("Test Accuracy :", test_acc)
print("Test F1 Score :", test_f1)
print("========================")

print("Model saved to:", MODEL_PATH)


Device: cuda
Classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite

In [8]:
# ================================================================
# ADSD-FAST (Option A: MAX SPEED) - Single cell for Kaggle
# - Diffusion disabled (fast mode)
# - Lightweight signature augmentation (no heavy UNet / diffusion)
# - Early stop when train_acc >= 0.96 AND val_acc >= 0.96 (or val >= 0.96)
# - Uses EfficientNet-B0 backbone (pretrained) and AMP
# - Outputs: saved model, training curves, confusion matrix, class report
# ================================================================

import os
import time
import random
from collections import Counter
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import transforms, datasets, models

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# ----------------------------
# Config (speed-optimized)
# ----------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# change these paths if your dataset is elsewhere
TRAIN_DIR = "/kaggle/input/leaf-dataset-vimal/dataset/train"
VALID_DIR = "/kaggle/input/leaf-dataset-vimal/dataset/valid"
TEST_DIR  = "/kaggle/input/leaf-dataset-vimal/dataset/test"

OUT_DIR = "/kaggle/working/adsd_fast"
MODEL_PATH = os.path.join(OUT_DIR, "ADSD_fast_model.pth")
os.makedirs(OUT_DIR, exist_ok=True)

CFG = {
    "image_size": 160,          # smaller = faster
    "batch_size": 32,
    "epochs": 40,               # but early stop will likely cut it short
    "lr": 2e-4,
    "weight_decay": 1e-5,
    "num_workers": 2,
    "target_acc": 0.96,         # 96% threshold
    "use_diffusion": False,     # Option A disables heavy diffusion
    "seed": 42
}

# reproducibility
torch.manual_seed(CFG["seed"])
np.random.seed(CFG["seed"])
random.seed(CFG["seed"])

# ----------------------------
# Lightweight Disease Signature Extractor (fast)
# ----------------------------
class TinySignatureExtractor(nn.Module):
    """Lightweight encoder that extracts a small disease signature vector"""
    def __init__(self, out_dim=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=2, padding=1),  # 80x80
            nn.BatchNorm2d(32), nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, stride=2, padding=1), # 40x40
            nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(64, out_dim),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.net(x)

# ----------------------------
# Lightweight augmenter using signatures (fast substitute for diffusion)
# ----------------------------
class LightweightAugmenter:
    def __init__(self, signature_extractor, device=DEVICE):
        self.sig = signature_extractor.to(device)
        self.device = device

    def augment(self, images, labels=None, strength=0.25):
        """
        images: tensor Bx3xHxW in [0,1] (we expect normalized later; here we receive pre-normalized tensors)
        returns augmented images (same shape)
        """
        with torch.no_grad():
            B = images.size(0)
            sig_vec = self.sig(images)  # B x D
            sv = (sig_vec - sig_vec.mean(dim=1, keepdim=True)) / (sig_vec.std(dim=1, keepdim=True) + 1e-6)
            D = sig_vec.size(1)
            maps = sv.unsqueeze(-1).unsqueeze(-1) * torch.ones((B, D, 1, 1), device=self.device)
            lesion_map = maps.mean(dim=1, keepdim=True)  # Bx1x1x1
            lesion_map = F.interpolate(lesion_map, size=(images.size(2), images.size(3)), mode="bilinear", align_corners=False)
            color_offsets = torch.tensor([0.9, 0.6, 0.6], device=self.device).view(1,3,1,1)
            lesion_rgb = lesion_map * color_offsets
            lesion_rgb = torch.tanh(lesion_rgb) * 0.5
            alpha = strength
            aug = (1-alpha) * images + alpha * (images + lesion_rgb)
            aug = torch.clamp(aug, 0.0, 1.0)
            return aug

# ----------------------------
# Fast classifier (EfficientNet-B0 backbone with small modifications)
# ----------------------------
class ADSD_FastClassifier(nn.Module):
    def __init__(self, num_classes, dropout=0.2):
        super().__init__()
        backbone = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        self.features = backbone.features
        feat_dim = backbone.classifier[1].in_features  # 1280
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(feat_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )
    def forward(self, x):
        f = self.features(x)
        f = self.pool(f).flatten(1)
        out = self.classifier(f)
        return out

# ----------------------------
# Transforms (fast)
# ----------------------------
def get_transforms(train=True):
    if train:
        return transforms.Compose([
            transforms.Resize((CFG["image_size"]+16, CFG["image_size"]+16)),
            transforms.RandomResizedCrop(CFG["image_size"], scale=(0.8,1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(p=0.1),
            transforms.ToTensor(),  # produces 0..1
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((CFG["image_size"], CFG["image_size"])),
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
        ])

# ----------------------------
# Helpers: class weights sampler
# ----------------------------
def get_class_weights(folder_dataset):
    labels = [lab for _, lab in folder_dataset.samples]
    counts = Counter(labels)
    total = len(labels)
    weights = [total / counts[lab] for _, lab in folder_dataset.samples]
    return weights

# ----------------------------
# Plotting helpers
# ----------------------------
def plot_training_curves(history, path):
    plt.figure(figsize=(10,3))
    plt.subplot(1,3,1); plt.plot(history['train_loss'], label='train'); plt.plot(history['val_loss'], label='val'); plt.title('Loss'); plt.legend()
    plt.subplot(1,3,2); plt.plot(history['train_acc'], label='train'); plt.plot(history['val_acc'], label='val'); plt.title('Accuracy'); plt.legend()
    plt.subplot(1,3,3); plt.plot(history['train_f1'], label='train'); plt.plot(history['val_f1'], label='val'); plt.title('F1'); plt.legend()
    plt.tight_layout(); plt.savefig(path); plt.close()

def plot_confusion(cm, classes, path):
    plt.figure(figsize=(10,8));
    import seaborn as sns
    sns.heatmap(cm, annot=False, cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xticks(rotation=90); plt.yticks(rotation=0)
    plt.tight_layout(); plt.savefig(path); plt.close()

# ----------------------------
# Training and evaluation functions
# ----------------------------
def train_one_epoch(model, loader, criterion, optimizer, scaler, augmenter=None, use_augment=False):
    model.train()
    total_loss = 0.0
    preds=[]; labs=[]
    for imgs, labels in loader:
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)
        # images come normalized; augment expects un-normalized [0,1] so we undo norm -> augment -> renorm
        if augmenter is not None and use_augment and torch.rand(1).item() < 0.5:
            mean = torch.tensor([0.485,0.456,0.406], device=DEVICE).view(1,3,1,1)
            std = torch.tensor([0.229,0.224,0.225], device=DEVICE).view(1,3,1,1)
            unnorm = imgs * std + mean
            aug = augmenter.augment(unnorm, labels, strength=0.2)
            imgs = (aug - mean) / std

        optimizer.zero_grad()
        with autocast():
            out = model(imgs)
            loss = criterion(out, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item() * imgs.size(0)
        preds.extend(out.argmax(1).detach().cpu().numpy().tolist())
        labs.extend(labels.detach().cpu().numpy().tolist())

    avg_loss = total_loss / len(loader.dataset)
    acc = accuracy_score(labs, preds)
    f1 = f1_score(labs, preds, average='weighted')
    return avg_loss, acc, f1

def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    preds=[]; labs=[]
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(DEVICE); labels = labels.to(DEVICE)
            out = model(imgs)
            loss = criterion(out, labels)
            total_loss += loss.item() * imgs.size(0)
            preds.extend(out.argmax(1).cpu().numpy().tolist())
            labs.extend(labels.cpu().numpy().tolist())
    avg_loss = total_loss / len(loader.dataset)
    acc = accuracy_score(labs, preds)
    f1 = f1_score(labs, preds, average='weighted')
    return avg_loss, acc, f1, preds, labs

# ----------------------------
# Main (single-cell)
# ----------------------------
def main():
    print("ADSD-FAST (Option A) starting. Device:", DEVICE)
    # Datasets
    train_ds = datasets.ImageFolder(TRAIN_DIR, transform=get_transforms(train=True))
    valid_ds = datasets.ImageFolder(VALID_DIR, transform=get_transforms(train=False))
    test_ds  = datasets.ImageFolder(TEST_DIR,  transform=get_transforms(train=False))

    classes = train_ds.classes
    print("Classes:", len(classes))

    # Weighted sampler for class imbalance
    weights = get_class_weights(train_ds)
    sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

    train_loader = DataLoader(train_ds, batch_size=CFG["batch_size"], sampler=sampler,
                              num_workers=CFG["num_workers"], pin_memory=True)
    valid_loader = DataLoader(valid_ds, batch_size=CFG["batch_size"], shuffle=False,
                              num_workers=CFG["num_workers"], pin_memory=True)
    test_loader  = DataLoader(test_ds,  batch_size=CFG["batch_size"], shuffle=False,
                              num_workers=CFG["num_workers"], pin_memory=True)

    # Models
    classifier = ADSD_FastClassifier(num_classes=len(classes), dropout=0.2).to(DEVICE)
    signature_extractor = TinySignatureExtractor(out_dim=48).to(DEVICE)
    augmenter = LightweightAugmenter(signature_extractor, device=DEVICE)

    # Optimizer / loss / scaler
    criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
    optimizer = torch.optim.AdamW(classifier.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])
    scaler = GradScaler()

    history = {'train_loss':[], 'val_loss':[], 'train_acc':[], 'val_acc':[], 'train_f1':[], 'val_f1':[]}
    best_val = 0.0

    start = time.time()
    for epoch in range(1, CFG["epochs"]+1):
        use_augment = epoch >= 2  # start augmentation after 1 epoch
        tloss, tacc, tf1 = train_one_epoch(classifier, train_loader, criterion, optimizer, scaler, augmenter, use_augment)
        vloss, vacc, vf1, vpreds, vlabels = evaluate(classifier, valid_loader, criterion)

        history['train_loss'].append(tloss); history['val_loss'].append(vloss)
        history['train_acc'].append(tacc); history['val_acc'].append(vacc)
        history['train_f1'].append(tf1); history['val_f1'].append(vf1)

        print(f"Epoch {epoch}/{CFG['epochs']}  Train Acc: {tacc:.4f}  Val Acc: {vacc:.4f}  Train Loss: {tloss:.4f}  Val Loss: {vloss:.4f}")

        # Save best
        if vacc > best_val + 1e-6:
            best_val = vacc
            torch.save({'model_state': classifier.state_dict(), 'classes': classes, 'cfg': CFG}, MODEL_PATH)
            print("  -> Saved best model (val improved).")

        # Early stop based on 96% threshold
        if tacc >= CFG["target_acc"] and vacc >= CFG["target_acc"]:
            print(f"\nEARLY STOP: train_acc ({tacc:.4f}) >= {CFG['target_acc']} AND val_acc ({vacc:.4f}) >= {CFG['target_acc']}")
            break
        if vacc >= CFG["target_acc"]:
            print(f"\nEARLY STOP: val_acc ({vacc:.4f}) >= {CFG['target_acc']}")
            break

    elapsed = time.time() - start
    print(f"\nTraining finished in {elapsed/60:.2f} minutes. Best val acc: {best_val:.4f}")

    # Load best model & evaluate on train/val/test
    ckpt = torch.load(MODEL_PATH, map_location=DEVICE)
    classifier.load_state_dict(ckpt['model_state'])
    classifier.to(DEVICE); classifier.eval()

    tr_loss, tr_acc, tr_f1, tr_preds, tr_labels = evaluate(classifier, train_loader, criterion)
    v_loss, v_acc, v_f1, v_preds, v_labels = evaluate(classifier, valid_loader, criterion)
    te_loss, te_acc, te_f1, te_preds, te_labels = evaluate(classifier, test_loader, criterion)

    print("\n=== FINAL RESULTS ===")
    print(f"Train Acc: {tr_acc:.4f}  Val Acc: {v_acc:.4f}  Test Acc: {te_acc:.4f}")
    print(f"Train F1 : {tr_f1:.4f}  Val F1 : {v_f1:.4f}  Test F1 : {te_f1:.4f}")

    # Save metrics and plots
    plot_training_curves(history, os.path.join(OUT_DIR, "training_curves.png"))
    cm = confusion_matrix(v_labels, v_preds)
    plot_confusion(cm, classes, os.path.join(OUT_DIR, "confusion_matrix_val.png"))

    # Classification report (validation)
    report = classification_report(v_labels, v_preds, target_names=classes, digits=4)
    with open(os.path.join(OUT_DIR, "classification_report_val.txt"), "w") as f:
        f.write(report)

    # Save final model (also saved best), also save inference helper files
    torch.save({"model_state": classifier.state_dict(), "classes": classes, "cfg": CFG}, os.path.join(OUT_DIR, "final_model.pth"))

    print("\nArtifacts saved to:", OUT_DIR)
    return classifier, classes

if __name__ == "__main__":
    main()


Device: cuda
ADSD-FAST (Option A) starting. Device: cuda
Classes: 38
Epoch 1/40  Train Acc: 0.7645  Val Acc: 0.9463  Train Loss: 1.3177  Val Loss: 0.5627
  -> Saved best model (val improved).
Epoch 2/40  Train Acc: 0.9601  Val Acc: 0.9789  Train Loss: 0.5440  Val Loss: 0.4881
  -> Saved best model (val improved).

EARLY STOP: train_acc (0.9601) >= 0.96 AND val_acc (0.9789) >= 0.96

Training finished in 0.81 minutes. Best val acc: 0.9789

=== FINAL RESULTS ===
Train Acc: 0.9919  Val Acc: 0.9789  Test Acc: 0.9737
Train F1 : 0.9919  Val F1 : 0.9787  Test F1 : 0.9734

Artifacts saved to: /kaggle/working/adsd_fast


In [10]:
# =====================================================================
#            S-ViT Lite FAST (MODIFIED VERSION) - FULL SCRIPT
#   Saves both separate files and a single unified file containing both.
# =====================================================================

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, WeightedRandomSampler, Dataset
from torch.cuda.amp import GradScaler, autocast
from torchvision import transforms, datasets
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from collections import Counter
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# =====================================================================
# CONFIG (EXTREME SPEED MODE)
# =====================================================================

CONFIG = {
    'train_dir': '/kaggle/input/leaf-dataset-vimal/dataset/train',   # update if needed
    'valid_dir': '/kaggle/input/leaf-dataset-vimal/dataset/valid',
    'test_dir':  '/kaggle/input/leaf-dataset-vimal/dataset/test',

    'image_size': 160,
    'patch_size': 16,
    'embed_dim': 256,
    'heads': 4,
    'layers': 3,
    'mlp_ratio': 3,
    'dropout': 0.1,

    'batch_size': 32,
    'epochs': 50,
    'learning_rate': 1e-4,
    'weight_decay': 1e-4,
    'num_workers': 2,

    'patience': 7,
    'target_acc': 0.96,

    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'save_dir': '/kaggle/working/SViT_Lite_Fast_Modified'
}

os.makedirs(CONFIG['save_dir'], exist_ok=True)
DEVICE = torch.device(CONFIG['device'])
print("DEVICE:", DEVICE)

# =====================================================================
# PSEUDO-NIR GENERATOR (FAST)
# =====================================================================

class PseudoNIR(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 1, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        r = x[:,0:1]; g = x[:,1:2]; b = x[:,2:3]
        veg = torch.clamp(2*g - r - b, 0, 1)
        return 0.7*self.net(x) + 0.3*veg

# =====================================================================
# SPECTRAL ATTENTION (FAST)
# =====================================================================

class SpectralAttention(nn.Module):
    def __init__(self, ch=4):
        super().__init__()
        self.channel_gate = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(ch, ch, 1),
            nn.Sigmoid()
        )
        self.spatial_gate = nn.Sequential(
            nn.Conv2d(ch, 1, 7, padding=3),
            nn.Sigmoid()
        )
    def forward(self, x):
        x = x * self.channel_gate(x)
        x = x * self.spatial_gate(x)
        return x

# =====================================================================
# PATCH EMBEDDING
# =====================================================================

class PatchEmbedding(nn.Module):
    def __init__(self, img=160, patch=16, ch=4, dim=256):
        super().__init__()
        self.num_patches = (img // patch) ** 2
        self.proj = nn.Conv2d(ch, dim, patch, patch)
    def forward(self, x):
        x = self.proj(x)
        return x.flatten(2).transpose(1, 2)

# =====================================================================
# TRANSFORMER BLOCK (LIGHT)
# =====================================================================

class MHA(nn.Module):
    def __init__(self, dim, heads=4, drop=0.1):
        super().__init__()
        self.attn = nn.MultiheadAttention(dim, heads, dropout=drop, batch_first=True)
        self.drop = nn.Dropout(drop)
    def forward(self, x):
        out, _ = self.attn(x, x, x)
        return x + self.drop(out)

class MLP(nn.Module):
    def __init__(self, dim, ratio=3, drop=0.1):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim*ratio)
        self.fc2 = nn.Linear(dim*ratio, dim)
        self.drop = nn.Dropout(drop)
    def forward(self, x):
        return x + self.drop(self.fc2(F.gelu(self.fc1(x))))

class Transformer(nn.Module):
    def __init__(self, dim, heads, ratio, drop):
        super().__init__()
        self.n1 = nn.LayerNorm(dim)
        self.att = MHA(dim, heads, drop)
        self.n2 = nn.LayerNorm(dim)
        self.mlp = MLP(dim, ratio, drop)
    def forward(self, x):
        x = self.att(self.n1(x))
        x = self.mlp(self.n2(x))
        return x

# =====================================================================
# S-ViT LITE FAST MODEL (MODIFIED)
# =====================================================================

class SViT_Lite_Fast_Modified(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.nir = PseudoNIR()
        self.satt = SpectralAttention(4)

        self.patch = PatchEmbedding(CONFIG['image_size'], CONFIG['patch_size'], 4, CONFIG['embed_dim'])
        N = self.patch.num_patches
        D = CONFIG['embed_dim']

        self.cls = nn.Parameter(torch.zeros(1, 1, D))
        self.pos = nn.Parameter(torch.zeros(1, N + 1, D))

        self.blocks = nn.ModuleList([
            Transformer(D, CONFIG['heads'], CONFIG['mlp_ratio'], CONFIG['dropout'])
            for _ in range(CONFIG['layers'])
        ])

        self.norm = nn.LayerNorm(D)
        self.head = nn.Linear(D, num_classes)

        nn.init.trunc_normal_(self.cls, std=0.02)
        nn.init.trunc_normal_(self.pos, std=0.02)

    def forward(self, x):
        nir = self.nir(x)
        x = torch.cat([x, nir], 1)
        x = self.satt(x)

        x = self.patch(x)
        B = x.size(0)

        cls = self.cls.expand(B, -1, -1)
        x = torch.cat([cls, x], 1)
        x = x + self.pos

        for blk in self.blocks:
            x = blk(x)

        x = self.norm(x)
        return self.head(x[:,0])

# =====================================================================
# DATASET + TRANSFORMS
# =====================================================================

def get_transforms(train=True):
    if train:
        return transforms.Compose([
            transforms.Resize((CONFIG["image_size"]+16, CONFIG["image_size"]+16)),
            transforms.RandomResizedCrop(CONFIG["image_size"], scale=(0.8,1)),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(0.2,0.2,0.2),
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((CONFIG["image_size"], CONFIG["image_size"])),
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
        ])

class SpectralDataset(Dataset):
    def __init__(self, root, transform=None):
        self.df = datasets.ImageFolder(root)
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        img, lbl = self.df[i]
        if self.transform: img = self.transform(img)
        return img, lbl
    @property
    def classes(self): return self.df.classes

def get_class_weights(ds):
    labels = [lbl for _, lbl in ds.df.samples]
    c = Counter(labels)
    total = len(labels)
    return [total / c[lbl] for _, lbl in ds.df.samples]

# =====================================================================
# TRAIN/EVAL
# =====================================================================

def train_epoch(model, loader, crit, opt, scaler):
    model.train()
    preds=[]; labs=[]; total_loss=0.0
    for x,y in loader:
        x,y = x.to(DEVICE), y.to(DEVICE)
        opt.zero_grad()
        with autocast():
            out = model(x)
            loss = crit(out, y)
        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()
        total_loss += loss.item() * x.size(0)
        preds += out.argmax(1).cpu().numpy().tolist()
        labs += y.cpu().numpy().tolist()
    avg_loss = total_loss / len(loader.dataset)
    acc = accuracy_score(labs, preds)
    f1 = f1_score(labs, preds, average='weighted')
    return avg_loss, acc, f1

def evaluate(model, loader, crit):
    model.eval()
    preds=[]; labs=[]; total_loss=0.0
    with torch.no_grad():
        for x,y in loader:
            x,y = x.to(DEVICE), y.to(DEVICE)
            out = model(x)
            loss = crit(out, y)
            total_loss += loss.item() * x.size(0)
            preds += out.argmax(1).cpu().numpy().tolist()
            labs += y.cpu().numpy().tolist()
    avg_loss = total_loss / len(loader.dataset)
    acc = accuracy_score(labs, preds)
    f1 = f1_score(labs, preds, average='weighted')
    return avg_loss, acc, f1, preds, labs

# =====================================================================
# PLOTTING HELPERS
# =====================================================================

def plot_curves(h, path):
    plt.figure(figsize=(10,3))
    plt.subplot(1,3,1); plt.plot(h['tl']); plt.plot(h['vl']); plt.title("Loss")
    plt.subplot(1,3,2); plt.plot(h['ta']); plt.plot(h['va']); plt.title("Acc")
    plt.subplot(1,3,3); plt.plot(h['tf']); plt.plot(h['vf']); plt.title("F1")
    plt.tight_layout(); plt.savefig(path); plt.close()

# =====================================================================
# MAIN
# =====================================================================

def main():
    print("\n=== LOADING DATA ===\n")
    train_ds = SpectralDataset(CONFIG['train_dir'], get_transforms(True))
    valid_ds = SpectralDataset(CONFIG['valid_dir'], get_transforms(False))
    test_ds  = SpectralDataset(CONFIG['test_dir'],  get_transforms(False))

    sampler = WeightedRandomSampler(get_class_weights(train_ds), len(train_ds))

    train_loader = DataLoader(train_ds, batch_size=CONFIG['batch_size'], sampler=sampler, num_workers=CONFIG['num_workers'], pin_memory=True)
    valid_loader = DataLoader(valid_ds, batch_size=CONFIG['batch_size'], shuffle=False, pin_memory=True)
    test_loader  = DataLoader(test_ds,  batch_size=CONFIG['batch_size'], shuffle=False, pin_memory=True)

    classes = train_ds.classes
    print("Classes:", len(classes))

    # Model
    model = SViT_Lite_Fast_Modified(len(classes)).to(DEVICE)
    print("Parameters:", sum(p.numel() for p in model.parameters()))

    crit = nn.CrossEntropyLoss(label_smoothing=0.1)
    opt  = torch.optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
    sched = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(opt, 10)
    scaler = GradScaler()

    history = {'tl':[], 'ta':[], 'tf':[], 'vl':[], 'va':[], 'vf':[]}
    best_val = 0.0
    patience = 0

    print("\n=== TRAINING STARTED ===\n")
    for epoch in range(CONFIG['epochs']):
        tr_loss, tr_acc, tr_f1 = train_epoch(model, train_loader, crit, opt, scaler)
        val_loss, val_acc, val_f1, val_preds, val_labels = evaluate(model, valid_loader, crit)
        sched.step()

        history['tl'].append(tr_loss); history['ta'].append(tr_acc); history['tf'].append(tr_f1)
        history['vl'].append(val_loss); history['va'].append(val_acc); history['vf'].append(val_f1)

        print(f"Epoch {epoch+1}/{CONFIG['epochs']}  Train Acc: {tr_acc:.4f}  Val Acc: {val_acc:.4f}  Train Loss: {tr_loss:.4f}  Val Loss: {val_loss:.4f}")

        # Save best checkpoint (validation-driven)
        if val_acc > best_val + 1e-8:
            best_val = val_acc
            patience = 0
            best_path = os.path.join(CONFIG['save_dir'], "S-ViT_Lite_Fast_Modified_best.pth")
            torch.save(model.state_dict(), best_path)
            print(f"  -> Saved best model (val_acc={val_acc:.4f}) to {best_path}")
        else:
            patience += 1

        # Early stop if target reached
        if (tr_acc >= CONFIG['target_acc'] and val_acc >= CONFIG['target_acc']) or (val_acc >= CONFIG['target_acc']):
            print(f"\n*** EARLY STOP: target {CONFIG['target_acc']*100:.0f}% reached (train={tr_acc:.4f}, val={val_acc:.4f}) ***\n")
            break

        if patience >= CONFIG['patience']:
            print(f"\nEarly stopping due to no improvement for {CONFIG['patience']} epochs.\n")
            break

    # After training, always save final state
    final_path = os.path.join(CONFIG['save_dir'], "S-ViT_Lite_Fast_Modified_final.pth")
    torch.save(model.state_dict(), final_path)
    print("Saved final model to:", final_path)

    # Ensure best model exists (if not, set best=model)
    best_path = os.path.join(CONFIG['save_dir'], "S-ViT_Lite_Fast_Modified_best.pth")
    if not os.path.exists(best_path):
        torch.save(model.state_dict(), best_path)
        print("No best checkpoint found – saved current final as best.")

    # Unified single-file save containing both best + final + metadata
    best_state = torch.load(best_path, map_location='cpu')
    final_state = torch.load(final_path, map_location='cpu')
    unified = {
        'best_model_state': best_state,
        'final_model_state': final_state,
        'classes': classes,
        'config': CONFIG,
        'best_val_acc': float(best_val)
    }
    unified_path = os.path.join(CONFIG['save_dir'], "S-ViT_Lite_Fast_Modified.pth")
    torch.save(unified, unified_path)
    print("Saved unified checkpoint containing best+final to:", unified_path)

    # Final evaluation on train/val/test using the best model (recommended for inference)
    model.load_state_dict(unified['best_model_state'])
    model.to(DEVICE).eval()

    tr_l, tr_acc, tr_f1, tr_preds, tr_labels = evaluate(model, train_loader, crit)
    v_l, v_acc, v_f1, v_preds, v_labels = evaluate(model, valid_loader, crit)
    te_l, te_acc, te_f1, te_preds, te_labels = evaluate(model, test_loader, crit)

    print("\n=== FINAL METRICS (using BEST checkpoint) ===")
    print(f"Train  Acc: {tr_acc:.4f}  F1: {tr_f1:.4f}")
    print(f"Valid  Acc: {v_acc:.4f}  F1: {v_f1:.4f}")
    print(f"Test   Acc: {te_acc:.4f}  F1: {te_f1:.4f}")

    # Save confusion matrix + classification report (validation)
    cm = confusion_matrix(v_labels, v_preds)
    plt.figure(figsize=(10,8)); plt.imshow(cm, cmap='Blues'); plt.title("Validation Confusion Matrix"); plt.colorbar()
    plt.tight_layout(); plt.savefig(os.path.join(CONFIG['save_dir'], "confusion_matrix.png")); plt.close()

    report = classification_report(v_labels, v_preds, target_names=classes, digits=4)
    with open(os.path.join(CONFIG['save_dir'], "classification_report_val.txt"), "w") as f:
        f.write(report)

    # Save training curves
    plot_curves(history, os.path.join(CONFIG['save_dir'], "training_curves.png"))

    print("\nAll artifacts saved in:", CONFIG['save_dir'])
    print("Files saved:")
    print(" -", best_path)
    print(" -", final_path)
    print(" -", unified_path)
    print("\nDone.")

if __name__ == "__main__":
    main()


DEVICE: cuda

=== LOADING DATA ===

Classes: 38
Parameters: 2274768

=== TRAINING STARTED ===

Epoch 1/50  Train Acc: 0.2705  Val Acc: 0.4205  Train Loss: 2.8744  Val Loss: 2.3664
  -> Saved best model (val_acc=0.4205) to /kaggle/working/SViT_Lite_Fast_Modified/S-ViT_Lite_Fast_Modified_best.pth
Epoch 2/50  Train Acc: 0.5116  Val Acc: 0.5805  Train Loss: 2.1055  Val Loss: 1.8535
  -> Saved best model (val_acc=0.5805) to /kaggle/working/SViT_Lite_Fast_Modified/S-ViT_Lite_Fast_Modified_best.pth
Epoch 3/50  Train Acc: 0.6162  Val Acc: 0.6437  Train Loss: 1.7827  Val Loss: 1.6707
  -> Saved best model (val_acc=0.6437) to /kaggle/working/SViT_Lite_Fast_Modified/S-ViT_Lite_Fast_Modified_best.pth
Epoch 4/50  Train Acc: 0.6906  Val Acc: 0.7400  Train Loss: 1.5743  Val Loss: 1.4372
  -> Saved best model (val_acc=0.7400) to /kaggle/working/SViT_Lite_Fast_Modified/S-ViT_Lite_Fast_Modified_best.pth
Epoch 5/50  Train Acc: 0.7463  Val Acc: 0.7605  Train Loss: 1.4388  Val Loss: 1.3633
  -> Saved best 

In [13]:
# ============================================================
# ECAM_fast (Option A) - Single Kaggle cell (NO visualizations)
# - Fast/resnet18 backbone
# - causal_dim = 128
# - num_regions = 25 (5x5)
# - mixed precision (AMP)
# - early stop when train, val and test all >= 95%
# - saves:
#     ECAM_fast_best.pth
#     ECAM_fast_final.pth
#     ECAM_fast.pth (unified)
# ============================================================

import os
import random
import warnings
from collections import Counter
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models
from torch.cuda.amp import autocast, GradScaler

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# -------------------------
# User dataset paths (Kaggle)
# -------------------------
TRAIN_DIR = "/kaggle/input/leaf-dataset-vimal/dataset/train"
VALID_DIR = "/kaggle/input/leaf-dataset-vimal/dataset/valid"
TEST_DIR  = "/kaggle/input/leaf-dataset-vimal/dataset/test"

# -------------------------
# Config (Option A: speed)
# -------------------------
CONFIG = {
    "image_size": 224,
    "batch_size": 48,          # reasonably large for T4; reduce if OOM
    "epochs": 20,
    "lr": 1e-4,
    "weight_decay": 1e-4,
    "num_regions": 25,         # 5x5
    "causal_dim": 128,
    "dropout": 0.3,
    "patience": 6,
    "target_accuracy": 0.95,   # 95%
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "num_workers": 2,
    "save_dir": "/kaggle/working/ECAM_fast"
}
os.makedirs(CONFIG["save_dir"], exist_ok=True)
DEVICE = torch.device(CONFIG["device"])
print("Device:", DEVICE)

# -------------------------
# Transforms & loaders
# -------------------------
def get_transforms(train=True):
    if train:
        return transforms.Compose([
            transforms.Resize((CONFIG["image_size"], CONFIG["image_size"])),
            transforms.RandomResizedCrop(CONFIG["image_size"], scale=(0.8,1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(0.1,0.1,0.05),
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((CONFIG["image_size"], CONFIG["image_size"])),
            transforms.CenterCrop(CONFIG["image_size"]),
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
        ])

train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=get_transforms(train=True))
valid_dataset = datasets.ImageFolder(VALID_DIR, transform=get_transforms(train=False))
test_dataset  = datasets.ImageFolder(TEST_DIR,  transform=get_transforms(train=False))

print("Classes:", train_dataset.classes)
print("Samples - train:", len(train_dataset), "valid:", len(valid_dataset), "test:", len(test_dataset))

train_loader = DataLoader(train_dataset, batch_size=CONFIG["batch_size"], shuffle=True,
                          num_workers=CONFIG["num_workers"], pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=CONFIG["batch_size"], shuffle=False,
                          num_workers=CONFIG["num_workers"], pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=CONFIG["batch_size"], shuffle=False,
                          num_workers=CONFIG["num_workers"], pin_memory=True)

# -------------------------
# Lightweight causal components (optimized)
# -------------------------
class CausalMaskGenerator(nn.Module):
    def __init__(self, in_ch, num_regions=25):
        super().__init__()
        grid = int(np.sqrt(num_regions))
        self.mask_net = nn.Sequential(
            nn.Conv2d(in_ch, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((grid, grid)),
            nn.Conv2d(64, 1, kernel_size=1),
            nn.Sigmoid()
        )
        self.causal_strength = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(in_ch, num_regions),
            nn.Softmax(dim=1)
        )

    def forward(self, features):
        mask = self.mask_net(features)                              # B x 1 x grid x grid
        strength = self.causal_strength(features).clamp(1e-8,1.0)   # B x num_regions
        return mask, strength

class CounterfactualInterventionModule:
    def __init__(self):
        pass

    def intervene(self, features, mask, mode='zero'):
        # features: B x C x H x W ; mask: B x 1 x gh x gw
        mask_resized = F.interpolate(mask, size=features.shape[2:], mode='bilinear', align_corners=False)
        if mode == 'zero':
            return features * (1 - mask_resized)
        elif mode == 'blur':
            blurred = F.avg_pool2d(features, kernel_size=3, stride=1, padding=1)
            return features * (1 - mask_resized) + blurred * mask_resized
        else:
            noise = torch.randn_like(features) * 0.05
            return features * (1 - mask_resized) + (features + noise) * mask_resized

class CausalGraphReasoning(nn.Module):
    def __init__(self, feat_dim, num_regions=25, hidden=128, top_k=6):
        super().__init__()
        self.num_regions = num_regions
        self.top_k = top_k
        self.node_lin = nn.Linear(feat_dim, hidden)
        self.edge_mlp = nn.Sequential(
            nn.Linear(hidden*2, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1),
            nn.Sigmoid()
        )
        self.msg = nn.Sequential(nn.Linear(hidden, hidden), nn.ReLU())
        self.out = nn.Linear(hidden, feat_dim)

    def forward(self, region_features):
        # region_features: B x N x D
        B, N, D = region_features.shape
        nodes = self.node_lin(region_features)            # B x N x H
        # compute pairwise scores efficiently
        # For efficiency: compute similarity matrix via dot product
        # nodes: B x N x H
        sim = torch.bmm(nodes, nodes.transpose(1,2))     # B x N x N
        # Keep top-k per node (excluding self)
        mask = torch.eye(N, device=sim.device).unsqueeze(0).expand(B, -1, -1) * -1e9
        sim = sim + mask
        topk_vals, topk_idx = sim.topk(min(self.top_k, N-1), dim=-1)  # B x N x K
        # Build aggregated messages
        gathered = []
        for k in range(topk_idx.size(-1)):
            idx = topk_idx[:,:,k]                                         # B x N
            batch_idx = torch.arange(B, device=idx.device).unsqueeze(-1).expand(-1, idx.size(1))
            neigh = nodes[batch_idx, idx]                                 # B x N x H
            gathered.append(neigh)
        neighs = torch.stack(gathered, dim=-2).mean(dim=-2)                # B x N x H
        msgs = self.msg(neighs)                                           # B x N x H
        updated = nodes + msgs
        out = self.out(updated)                                           # B x N x D
        return out, topk_vals

# -------------------------
# ECAM_fast model with ResNet18 backbone
# -------------------------
class ECAM_fast(nn.Module):
    def __init__(self, num_classes, num_regions=25, causal_dim=128, dropout=0.3):
        super().__init__()
        # ResNet18 backbone (fast)
        backbone = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        # remove last layers
        self.features = nn.Sequential(*list(backbone.children())[:-2])    # output C x H x W, C=512
        feat_ch = 512

        # lightweight reducer to causal_dim
        self.reduce = nn.Sequential(
            nn.Conv2d(feat_ch, causal_dim, kernel_size=1),
            nn.BatchNorm2d(causal_dim),
            nn.ReLU(inplace=True)
        )

        self.grid = int(np.sqrt(num_regions))
        self.num_regions = num_regions

        # causal modules
        self.mask_gen = CausalMaskGenerator(causal_dim, num_regions=num_regions)
        self.intervenor = CounterfactualInterventionModule()
        self.causal_graph = CausalGraphReasoning(causal_dim, num_regions=num_regions, hidden=causal_dim//2, top_k=6)

        # classification heads
        self.main_head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(causal_dim, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, num_classes)
        )
        self.causal_head = nn.Sequential(
            nn.Linear(causal_dim * num_regions, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, num_classes)
        )

    def forward(self, x, return_explanations=False):
        B = x.size(0)
        feat = self.features(x)                 # B x C x H x W
        feat = self.reduce(feat)                # B x causal_dim x h x w

        # Mask + strength
        mask, strength = self.mask_gen(feat)    # mask: B x1 x gh x gw ; strength: B x N

        # apply mask (attended features)
        attended = feat * F.interpolate(mask, size=feat.shape[2:], mode='bilinear', align_corners=False)

        # region pooling -> B x N x D
        pooled = F.adaptive_avg_pool2d(attended, (self.grid, self.grid))          # B x D x gh x gw
        B, D, gh, gw = pooled.shape
        regions = pooled.view(B, D, -1).permute(0,2,1).contiguous()              # B x N x D

        # causal graph reasoning
        causal_feats, edge_info = self.causal_graph(regions)                     # B x N x D

        # heads
        main_logits = self.main_head(attended)                                   # B x C
        causal_flat = causal_feats.view(B, -1)
        causal_logits = self.causal_head(causal_flat)                            # B x C

        logits = 0.75 * main_logits + 0.25 * causal_logits

        if return_explanations:
            return logits, {
                "mask": mask,
                "strength": strength,
                "edge_info": edge_info,
                "attended": attended
            }
        return logits

    def get_causal_explanation(self, x):
        self.eval()
        with torch.no_grad():
            logits, expl = self.forward(x, return_explanations=True)
            pred = logits.argmax(dim=1)
            conf = F.softmax(logits, dim=1).max(dim=1)[0]
        return {"prediction": pred, "confidence": conf, **expl}

# -------------------------
# Losses & training utilities
# -------------------------
def causal_consistency_loss(model, images, labels, criterion):
    # original
    model.eval()
    with torch.no_grad():
        logits, expl = model(images, return_explanations=True)
    model.train()

    # original loss (hard)
    orig_loss = criterion(logits, labels)

    # intervene (zero out causal mask)
    mask = expl["mask"]
    intervened_feat = model.intervenor.intervene(model.reduce(model.features(images)), mask, mode='zero')
    # run main head on intervened features (use classifier part only)
    main_logits = model.main_head(intervened_feat)
    # encourage divergence: KL(original || intervened)
    kl = F.kl_div(F.log_softmax(main_logits, dim=1), F.softmax(logits.detach(), dim=1), reduction='batchmean')
    # combine
    loss = orig_loss + 0.1 * kl
    return loss

def train_one_epoch(model, loader, optimizer, scaler, criterion, device, use_causal=False):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    for imgs, labels in loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        with autocast():
            if use_causal:
                loss = causal_consistency_loss(model, imgs, labels, criterion)
            else:
                out = model(imgs)
                loss = criterion(out, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * imgs.size(0)

        # predictions for metrics (compute under no_grad to avoid extra graph)
        with torch.no_grad():
            out = model(imgs)
            preds = out.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

    avg_loss = running_loss / len(loader.dataset)
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return avg_loss, acc, f1

def evaluate(model, loader, criterion, device):
    model.eval()
    preds = []
    labs = []
    loss_sum = 0.0
    cnt = 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            out = model(imgs)
            loss = criterion(out, labels)
            loss_sum += loss.item() * imgs.size(0)
            cnt += imgs.size(0)
            preds.extend(out.argmax(dim=1).cpu().numpy())
            labs.extend(labels.cpu().numpy())
    if cnt == 0: 
        return 0.0, 0.0, 0.0, [], []
    acc = accuracy_score(labs, preds)
    f1 = f1_score(labs, preds, average='weighted')
    return loss_sum / cnt, acc, f1, preds, labs

# -------------------------
# Prepare model / optimizer / scaler
# -------------------------
num_classes = len(train_dataset.classes)
model = ECAM_fast(num_classes=num_classes, num_regions=CONFIG["num_regions"],
                  causal_dim=CONFIG["causal_dim"], dropout=CONFIG["dropout"]).to(DEVICE)

# Use only a little weight decay, use AdamW
optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG["lr"], weight_decay=CONFIG["weight_decay"])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max(1, CONFIG["epochs"]))
scaler = GradScaler()
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)

print("Model params:", sum(p.numel() for p in model.parameters()))

# -------------------------
# Training loop with early stopping (monitor val, but also require train+val+test >= target to stop)
# -------------------------
history = {"train_loss":[], "train_acc":[], "train_f1":[], "val_loss":[], "val_acc":[], "val_f1":[], "test_acc":[]}
best_val = -1.0
patience = 0
best_state = None
final_state = None

for epoch in range(CONFIG["epochs"]):
    use_causal = epoch >= 3   # enable causal consistency loss after a couple epochs
    train_loss, train_acc, train_f1 = train_one_epoch(model, train_loader, optimizer, scaler, criterion, DEVICE, use_causal)
    val_loss, val_acc, val_f1, _, _ = evaluate(model, valid_loader, criterion, DEVICE)
    test_loss, test_acc, test_f1, _, _ = evaluate(model, test_loader, criterion, DEVICE)

    scheduler.step()

    history["train_loss"].append(train_loss); history["train_acc"].append(train_acc); history["train_f1"].append(train_f1)
    history["val_loss"].append(val_loss); history["val_acc"].append(val_acc); history["val_f1"].append(val_f1)
    history["test_acc"].append(test_acc)

    print(f"Epoch {epoch+1}/{CONFIG['epochs']}  Train acc: {train_acc:.4f}  Val acc: {val_acc:.4f}  Test acc: {test_acc:.4f}")

    # save best by validation acc
    if val_acc > best_val + 1e-8:
        best_val = val_acc
        patience = 0
        best_state = model.state_dict()
        torch.save(best_state, os.path.join(CONFIG["save_dir"], "ECAM_fast_best.pth"))
        print(f"  -> saved best (val acc {val_acc:.4f})")
    else:
        patience += 1

    # early stop if all three >= target
    if train_acc >= CONFIG["target_accuracy"] and val_acc >= CONFIG["target_accuracy"] and test_acc >= CONFIG["target_accuracy"]:
        print(f"\n*** EARLY STOP: reached target {CONFIG['target_accuracy']*100:.1f}% on train, val, test")
        final_state = model.state_dict()
        break

    # patience early stop
    if patience >= CONFIG["patience"]:
        print("\nEarly stopping due to patience")
        final_state = model.state_dict()
        break

# if not set, fill final_state with last model
if final_state is None:
    final_state = model.state_dict()
if best_state is None:
    best_state = final_state
    torch.save(best_state, os.path.join(CONFIG["save_dir"], "ECAM_fast_best.pth"))

# save final and unified
torch.save(final_state, os.path.join(CONFIG["save_dir"], "ECAM_fast_final.pth"))
unified = {
    "best_state": best_state,
    "final_state": final_state,
    "classes": train_dataset.classes,
    "config": CONFIG
}
torch.save(unified, os.path.join(CONFIG["save_dir"], "ECAM_fast.pth"))
print("Saved checkpoints to", CONFIG["save_dir"])

# final evaluation on best checkpoint
model.load_state_dict(best_state)
_, train_acc, train_f1, _, _ = evaluate(model, train_loader, criterion, DEVICE)
_, val_acc, val_f1, val_preds, val_labels = evaluate(model, valid_loader, criterion, DEVICE)
_, test_acc, test_f1, test_preds, test_labels = evaluate(model, test_loader, criterion, DEVICE)

print("\n=== FINAL (BEST MODEL) METRICS ===")
print(f"Train Acc: {train_acc:.4f}, F1: {train_f1:.4f}")
print(f"Val   Acc: {val_acc:.4f}, F1: {val_f1:.4f}")
print(f"Test  Acc: {test_acc:.4f}, F1: {test_f1:.4f}")

# save confusion matrix and classification report for validation
cm = confusion_matrix(val_labels, val_preds)
plt.figure(figsize=(10,10)); plt.imshow(cm, cmap="Blues"); plt.title("Val Confusion Matrix"); plt.colorbar()
plt.tight_layout(); plt.savefig(os.path.join(CONFIG["save_dir"], "confusion_matrix_val.png")); plt.close()

report = classification_report(val_labels, val_preds, target_names=train_dataset.classes, digits=4)
with open(os.path.join(CONFIG["save_dir"], "classification_report_val.txt"), "w") as f:
    f.write(report)

print("Artifacts saved (confusion matrix, classification report). Done.")


Device: cuda
Classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 184MB/s]


Model params: 12368167
Epoch 1/20  Train acc: 0.6485  Val acc: 0.9105  Test acc: 0.9232
  -> saved best (val acc 0.9105)
Epoch 2/20  Train acc: 0.9644  Val acc: 0.9658  Test acc: 0.9647
  -> saved best (val acc 0.9658)

*** EARLY STOP: reached target 95.0% on train, val, test
Saved checkpoints to /kaggle/working/ECAM_fast

=== FINAL (BEST MODEL) METRICS ===
Train Acc: 0.9796, F1: 0.9796
Val   Acc: 0.9658, F1: 0.9657
Test  Acc: 0.9647, F1: 0.9646
Artifacts saved (confusion matrix, classification report). Done.
